### Import data into database

In [15]:
from activity_tracker import utils
import sqlite3
import pandas as pd
import pathlib
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

# Import raw data into raw schema
data_dir = pathlib.Path("../data/raw/")
db_path = "../data/sqlite/activity_tracker.db"
mapper_path = pathlib.Path("../data/data_mapper.yml")
excel_path = pathlib.Path("../data/raw/MDE clinical data.xlsx")
mde_study_log_path = pathlib.Path("../data/raw/mde_study_log.csv")
excluded_files = [
    "fitbitBreathingRate_merged.csv"
    "fitbitCoreTemperature_merged.csv",
    "fitbitSkinTemperature_merged.csv",
    "mde_clinical_data.csv",
    "dailyCardioFitnessScore_merged.csv"
    "weightLogInfo_merged.csv",
    
]

### Create activity_tracker database

In [27]:
def camel_to_snake(name: str) -> str:
    name = name.replace("_merged", "")
    return re.sub(r'(?<=[a-z0-9])([A-Z])', r'_\1', name).lower()

conn = sqlite3.connect(db_path)

for file_path in data_dir.glob("*.csv"):
    if file_path.name in excluded_files:
        continue

    table_name = camel_to_snake(file_path.stem)
    df = pd.read_csv(file_path)
    
    # Convert date and time columns to datetime
    for col in df.columns:
        col_lower = col.lower()
        if ("date" in col_lower or "day" in col_lower):
            df[col] = pd.to_datetime(df[col]).dt.date 
        elif col_lower == "time":
            df[col] = pd.to_datetime(df[col])
    try:
        df.to_sql(table_name, conn, if_exists="fail", index=False)
        print(f"Table {table_name} created with {len(df)} rows.")
    except ValueError:
        print(f"Skipping {table_name} — table already exists.")
conn.close()

Table fitbit_wear_time_via_hr created with 140600 rows.
Table daily_activity created with 14271 rows.
Table sleep_day created with 1475 rows.
Table fitbit_daily_hrv created with 3 rows.
Table heart_rate_zones created with 52256 rows.
Table fitbit_daily_sp_o2 created with 1160 rows.
Table fitbit_breathing_rate created with 3 rows.
Table daily_calories created with 14271 rows.
Table daily_steps created with 14271 rows.
Table daily_cardio_fitness_score created with 690 rows.
Table sleep_stages_day created with 9178 rows.
Table mde_study_log created with 1000 rows.
Table daily_fitbit_active_zone_minutes created with 2104 rows.
Table daily_intensities created with 14271 rows.


### Create a subject table

In [28]:
xls = pd.ExcelFile(excel_path)
df_control = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
df_exercise = pd.read_excel(xls, sheet_name=xls.sheet_names[1])

# Add group labels
df_control["group"] = "control"
df_exercise["group"] = "exercise"
df_combined = pd.concat([df_control, df_exercise], ignore_index=True)

subject_columns = [
    "Participant ID", "group", "Sex", "Age", "Eth", "Race",
    "Mth Inc", "Educ", "Mari", "Liv Sit"
]
df_subject = df_combined[subject_columns].copy()

df_subject.columns = (
    df_subject.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(".", "_")
)

# Rename columns
rename_map = {
    "participant_id": "subject_id",
    "eth": "ethnicity",
    "mth_inc": "monthly_income",
    "educ": "education",
    "mari": "marital_status",
    "liv_sit": "living_situation"
}
df_subject = df_subject.rename(columns=rename_map)
df_subject = df_subject.sort_values(by="subject_id").reset_index(drop=True)

data_dictionary = utils.load_data_mapper(mapper_path)

# Apply data dictionary to map integer values to strings
for column in data_dictionary:
    if column in df_subject.columns:
        df_subject[column] = df_subject[column].map(data_dictionary[column])
        
conn = sqlite3.connect(db_path)
df_subject.to_sql("subject", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

In [29]:
df_subject

,subject_id,group,sex,age,ethnicity,race,monthly_income,education,marital_status,living_situation
0,MDE01,control,m,83.0,non_hisp,black,3000.0,18.0,never_married,with_others
1,MDE02,exercise,m,61.0,non_hisp,white,1033.0,3.0,never_married,alone
2,MDE03,exercise,f,69.0,hisp,white,1349.0,13.0,widowed,with_family
3,MDE04,control,f,65.0,non_hisp,black,900.0,11.0,widowed,with_family
4,MDE05,control,f,84.0,non_hisp,black,792.0,3.0,widowed,with_family
...,...,...,...,...,...,...,...,...,...,...
84,MDE85,control,f,90.0,hisp,white,NaN,6.0,widowed,with_family
85,MDE86,control,f,85.0,hisp,white,0.0,6.0,widowed,with_family
86,MDE87,control,f,99.0,hisp,unknown,0.0,3.0,widowed,with_family
87,MDE88,control,m,71.0,hisp,unknown,700.0,2.0,married,with_family


### Create a visit table

In [47]:
def make_unique_columns(cols):
    seen = {}
    result = []
    for col in cols:
        base = col
        if base not in seen:
            seen[base] = 1
            result.append(base)
        else:
            count = seen[base]
            new_col = f"{base}_{count}"
            while new_col in seen:
                count += 1
                new_col = f"{base}_{count}"
            seen[base] = count + 1
            seen[new_col] = 1
            result.append(new_col)
    return result

def sanitize_column(col):
    base = col.split(".")[0]
    # Remove anything in parentheses (and the parentheses themselves)
    base = re.sub(r"\(.*?\)", "", base)
    # Normalize spacing and other characters
    name = "_".join(base.strip().split()).lower()
    # Remove remaining special characters (slashes, dashes, percent signs)
    name = name.replace("/", "").replace("-", "").replace("%", "")
    return name

def stack_visits(df, group_label):
    df = df.rename(columns={"Participant ID": "subject_id"})
    visit_markers = sorted([col for col in df.columns if col.startswith("V") and len(col) == 2 and col[1].isdigit()],
                           key=lambda x: int(x[1:]))
    
    all_visits = []

    for i, marker in enumerate(visit_markers):
        start = df.columns.get_loc(marker) + 1
        end = df.columns.get_loc(visit_markers[i + 1]) if i + 1 < len(visit_markers) else len(df.columns)
        visit_cols = df.columns[start:end].tolist()

        visit_df = df[["subject_id"] + visit_cols].copy()
        cleaned_cols = [sanitize_column(col) for col in visit_cols]
        visit_df.columns = ["subject_id"] + make_unique_columns(cleaned_cols)

        visit_df.insert(1, "group", group_label)
        visit_df.insert(2, "visit_id", int(marker[1:]))

        all_visits.append(visit_df)

    return pd.concat(all_visits, ignore_index=True)

# Load data
xls = pd.ExcelFile(excel_path)

df_visits_control = stack_visits(pd.read_excel(xls, sheet_name=0), "control")
df_visits_exercise = stack_visits(pd.read_excel(xls, sheet_name=1), "exercise")

ffp_cols = ["wt_loss", "weak", "slow", "exhaust", "phys_act"]
for df in [df_visits_control, df_visits_exercise]:
    df["ffp_score"] = df[ffp_cols].sum(axis=1)
    df["gait"] = 4 / df["walk"].replace(0, pd.NA)

# Combine
df = pd.concat([df_visits_control, df_visits_exercise], ignore_index=True)
df = df.sort_values(by=["subject_id", "visit_id"]).reset_index(drop=True)
df = df[["subject_id", "group", "visit_id", "ffp_status", "ffp_score"]]
df["visit_id"] = df["visit_id"].astype("category")
df["ffp_status"] = df["ffp_status"].apply(lambda x: int(x) if pd.notnull(x) else pd.NA).astype("category")
df["ffp_score"] = df["ffp_score"].astype(float)

### Process study log

In [48]:
study_log_df = pd.read_csv(mde_study_log_path)

# Normalize subject IDs
study_log_df["subject_id"] = study_log_df["PID"].astype(str).str.replace("-", "").str.upper()

visit_date_cols = {
    "Screening": 1,
    "Week 4 (V2)": 2,
    "Week 8 (V3)": 3,
    "Week 12 (V4)": 4,
    "Week 24 (EOS)": 5
}
visit_dates_df = study_log_df[["subject_id"] + list(visit_date_cols.keys())].rename(columns=visit_date_cols)
visit_dates_df = visit_dates_df.dropna(subset=list(visit_date_cols.values()), how="all")

# Convert all visit date columns to datetime safely
for col in visit_date_cols.values():
    visit_dates_df[col] = pd.to_datetime(visit_dates_df[col].astype(str).str.strip().replace("OS", pd.NA), errors='coerce')

visit_dates_df

,subject_id,1,2,3,4,5
0,MDE01,2021-07-19,2021-08-23,2021-09-15,2021-10-07,2022-01-19
1,MDE02,2021-07-21,2021-08-30,2021-09-23,2021-10-13,2022-01-19
2,MDE03,2021-07-21,2021-08-30,2021-09-22,2021-10-11,2022-01-19
3,MDE04,2021-08-24,2021-09-22,NaT,NaT,NaT
4,MDE05,2021-09-16,2021-10-18,2021-11-11,2021-12-08,2022-02-14
...,...,...,...,...,...,...
96,MDE97,2024-08-15,2024-10-02,2024-10-28,2024-12-04,2025-02-27
97,MDE98,2024-10-14,2024-11-26,2025-01-07,2025-02-11,2025-05-06
98,MDE99,2024-10-23,NaT,NaT,NaT,NaT
99,MDE100,2024-10-24,2024-12-05,2025-01-07,2025-01-30,2025-05-01


In [49]:
# Merge visit dates with df
visit_dates_df = visit_dates_df.melt(
    id_vars="subject_id",
    var_name="visit_id",
    value_name="date"
)
df = df.merge(visit_dates_df, on=["subject_id", "visit_id"], how="left")
df["date"] = df["date"].dt.date
df

,subject_id,group,visit_id,ffp_status,ffp_score,date
0,MDE01,control,1,0,4.0,2021-07-19
1,MDE01,control,2,0,4.0,2021-08-23
2,MDE01,control,3,1,2.0,2021-09-15
3,MDE01,control,4,0,4.0,2021-10-07
4,MDE01,control,5,0,3.0,2022-01-19
...,...,...,...,...,...,...
440,MDE89,control,1,1,1.0,2023-11-01
441,MDE89,control,2,NaN,0.0,2023-12-29
442,MDE89,control,3,1,2.0,2024-01-25
443,MDE89,control,4,1,1.0,2024-02-29


In [50]:
# Create visit database
conn = sqlite3.connect(db_path)
df.to_sql("visit", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

### Create daily data table

In [34]:
def camel_to_snake(name):
    name = re.sub(r'(?<=[a-z0-9])([A-Z])', r'_\1', name)
    name = name.replace(" ", "_").replace("-", "_").replace("/", "_")
    return name.lower()

query = """
SELECT *
FROM fitbit_wear_time_via_hr wt
  LEFT JOIN daily_activity da
    ON wt.ID = da.ID AND wt.Day = da.ActivityDate
LEFT JOIN daily_intensities di
  ON da.ID = di.ID AND da.ActivityDate = di.ActivityDay
LEFT JOIN fitbit_daily_sp_o2 fb
  ON da.ID = fb.ID AND da.ActivityDate = fb.SleepDay
"""

conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()

# Column name sanitization
df.columns = [camel_to_snake(col) for col in df.columns]
df = df.loc[:, ~df.columns.duplicated()]
df = df.drop(columns=["activity_date", "activity_day", "sleep_day"]).rename({"id": "subject_id", "day": "date"}, axis=1)

In [35]:
conn = sqlite3.connect(db_path)
df.to_sql("daily_measurement", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

In [36]:
df.columns

Index(['subject_id', 'date', 'total_minutes_wear_time', 'percentage_wear_time',
       'total_steps', 'total_distance', 'tracker_distance',
       'logged_activities_distance', 'very_active_distance',
       'moderately_active_distance', 'light_active_distance',
       'sedentary_active_distance', 'very_active_minutes',
       'fairly_active_minutes', 'lightly_active_minutes', 'sedentary_minutes',
       'calories', 'floors', 'calories_bmr', 'marginal_calories',
       'resting_heart_rate', 'average_sp_o2', 'min_sp_o2', 'max_sp_o2'],
      dtype='object')

### Create analysis db

In [52]:
analysis_db_path = "../data/sqlite/analysis.db"
conn = sqlite3.connect(analysis_db_path)

conn.execute(f"ATTACH DATABASE '{db_path}' AS raw")
conn.execute("CREATE TABLE subject AS SELECT * FROM raw.subject")
conn.execute("CREATE TABLE visit AS SELECT * FROM raw.visit")
conn.execute("CREATE TABLE daily_measurement AS SELECT * FROM raw.daily_measurement")

conn.commit()
conn.close()

### Quality check

In [ ]:
# Filter rows
start_col = df.columns.get_loc("percentage_wear_time") + 1
right_cols = df.columns[start_col:]

# Drop rows where all right-side values are NaN
filtered_df = df[~df[right_cols].isna().all(axis=1)].reset_index(drop=True)
filtered_df

,subject_id,date,total_minutes_wear_time,percentage_wear_time,total_steps,total_distance,tracker_distance,logged_activities_distance,very_active_distance,moderately_active_distance,...,lightly_active_minutes,sedentary_minutes,calories,floors,calories_bmr,marginal_calories,resting_heart_rate,average_sp_o2,min_sp_o2,max_sp_o2
0,MDE01,2021-07-19,85,5.902778,147.0,0.1100,0.1100,0.0,0.0000,0.0,...,0.0,1440.0,1489.0,None,1489.0,0.0,NaN,NaN,NaN,NaN
1,MDE01,2021-07-20,630,43.750000,12.0,0.0100,0.0100,0.0,0.0000,0.0,...,0.0,1440.0,1473.0,None,1457.0,0.0,NaN,NaN,NaN,NaN
2,MDE01,2021-07-21,0,0.000000,0.0,0.0000,0.0000,0.0,0.0000,0.0,...,0.0,1440.0,1425.0,None,1425.0,0.0,NaN,NaN,NaN,NaN
3,MDE01,2021-07-22,0,0.000000,0.0,0.0000,0.0000,0.0,0.0000,0.0,...,0.0,1440.0,1425.0,None,1425.0,0.0,NaN,NaN,NaN,NaN
4,MDE01,2021-07-23,0,0.000000,0.0,0.0000,0.0000,0.0,0.0000,0.0,...,0.0,1440.0,1425.0,None,1425.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14266,MDE99,2024-10-19,13,0.902778,13.0,0.0000,0.0093,0.0,0.0000,0.0,...,2.0,1438.0,1462.0,None,1456.0,5.0,NaN,NaN,NaN,NaN
14267,MDE99,2024-10-20,44,3.055556,478.0,0.3094,0.3446,0.0,0.1229,0.0,...,20.0,1420.0,1503.0,None,1456.0,41.0,NaN,NaN,NaN,NaN
14268,MDE99,2024-10-21,5,0.347222,0.0,0.0000,0.0000,0.0,0.0000,0.0,...,0.0,1440.0,1456.0,None,1456.0,0.0,NaN,NaN,NaN,NaN
14269,MDE99,2024-10-22,9,0.625000,34.0,0.0245,0.0245,0.0,0.0000,0.0,...,3.0,1352.0,1377.0,None,1370.0,6.0,NaN,NaN,NaN,NaN


In [54]:
measure_days_df = filtered_df.groupby('subject_id').size().reset_index(name='measured_days').sort_values(by='measured_days', ascending=False)

# # Suppose you have a DataFrame called measure_days_df
# measure_days_df.to_sql('measure_days_temp', conn, if_exists='replace', index=False)

100